In [403]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
import sklearn
from sklearn.preprocessing import LabelEncoder


In [404]:
from bs4 import BeautifulSoup as soup
#from bs4 import requests

In [405]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score, train_test_split

In [406]:
df = pd.read_csv(r"C:\Users\sampf\OneDrive\Documents\Kaggle Real Estate Data USA\realtor-data.zip.csv")
df.head()


,brokered_by,status,price,bed,bath,acre_lot,street,city,state,zip_code,house_size,prev_sold_date
0,103378.0,for_sale,105000.0,3.0,2.0,0.12,1962661.0,Adjuntas,Puerto Rico,601.0,920.0,NaN
1,52707.0,for_sale,80000.0,4.0,2.0,0.08,1902874.0,Adjuntas,Puerto Rico,601.0,1527.0,NaN
2,103379.0,for_sale,67000.0,2.0,1.0,0.15,1404990.0,Juana Diaz,Puerto Rico,795.0,748.0,NaN
3,31239.0,for_sale,145000.0,4.0,2.0,0.10,1947675.0,Ponce,Puerto Rico,731.0,1800.0,NaN
4,34632.0,for_sale,65000.0,6.0,2.0,0.05,331151.0,Mayaguez,Puerto Rico,680.0,NaN,NaN


In [407]:
# Convert status to 3 binary columns, for_sale, ready_to_build, sold
dummy_var = pd.get_dummies(df['status'])
dummy_var
#Add above columns to our dataframe
#df = pd.concat([df, dummy_var])
#drop status column
df.drop('status', axis = 1, inplace = True)
df = df.loc[df['state'] == 'New York']
df.head()
# Replace True/False with binary values
df.replace(True, 1, inplace = True)
df.replace(False, 0, inplace = True)

# Check binary values are now in place in true/false columns
#print(df['for_sale'].value_counts())
#print(df['ready_to_build'].value_counts())
#print(df['sold'].value_counts())

In [408]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 103159 entries, 5217 to 1523273
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   brokered_by     103028 non-null  float64
 1   price           103157 non-null  float64
 2   bed             86911 non-null   float64
 3   bath            89954 non-null   float64
 4   acre_lot        73870 non-null   float64
 5   street          102181 non-null  float64
 6   city            103146 non-null  object 
 7   state           103159 non-null  object 
 8   zip_code        103138 non-null  float64
 9   house_size      72965 non-null   float64
 10  prev_sold_date  68057 non-null   object 
dtypes: float64(8), object(3)
memory usage: 9.4+ MB


In [409]:
df.describe()

,brokered_by,price,bed,bath,acre_lot,street,zip_code,house_size
count,103028.000000,1.031570e+05,86911.000000,89954.000000,73870.000000,1.021810e+05,103138.000000,72965.000000
mean,47556.979355,9.437968e+05,3.238796,2.304256,72.404475,9.702402e+05,12095.353972,1977.246214
std,28624.139690,3.672641e+06,1.932273,1.600848,1951.820181,5.533160e+05,1526.592075,2272.208046
min,32.000000,0.000000e+00,1.000000,1.000000,0.000000,5.500000e+01,4715.000000,4.000000
25%,23592.000000,1.825000e+05,2.000000,1.000000,0.110000,5.100950e+05,11206.000000,1150.000000
50%,42758.500000,4.200000e+05,3.000000,2.000000,0.250000,9.586430e+05,11934.000000,1618.000000
75%,67986.000000,8.490000e+05,4.000000,3.000000,1.000000,1.437630e+06,13203.000000,2300.000000
max,110111.000000,8.750000e+08,142.000000,123.000000,100000.000000,2.001247e+06,99999.000000,352836.000000


In [410]:
null_count = df.isnull().sum()
print(null_count)
df.head()


brokered_by         131
price                 2
bed               16248
bath              13205
acre_lot          29289
street              978
city                 13
state                 0
zip_code             21
house_size        30194
prev_sold_date    35102
dtype: int64


,brokered_by,price,bed,bath,acre_lot,street,city,state,zip_code,house_size,prev_sold_date
5217,10368.0,175000.0,3.0,1.0,60.00,553526.0,Berlin,New York,12022.0,1176.0,NaN
7970,22006.0,425000.0,3.0,2.0,2.02,263302.0,Claverack,New York,12521.0,1600.0,2021-11-24
7975,48310.0,225000.0,4.0,2.0,0.24,871278.0,Copake,New York,12521.0,1239.0,2018-02-01
7976,7797.0,419000.0,3.0,3.0,1.90,286373.0,Copake,New York,12516.0,1800.0,NaN
7977,23402.0,365000.0,3.0,2.0,2.00,78544.0,Copake,New York,12517.0,1482.0,NaN


In [411]:
df_corr = df.drop(['city', 'state', 'prev_sold_date'], axis = 1, inplace = False)
df_corr.head()

,brokered_by,price,bed,bath,acre_lot,street,zip_code,house_size
5217,10368.0,175000.0,3.0,1.0,60.00,553526.0,12022.0,1176.0
7970,22006.0,425000.0,3.0,2.0,2.02,263302.0,12521.0,1600.0
7975,48310.0,225000.0,4.0,2.0,0.24,871278.0,12521.0,1239.0
7976,7797.0,419000.0,3.0,3.0,1.90,286373.0,12516.0,1800.0
7977,23402.0,365000.0,3.0,2.0,2.00,78544.0,12517.0,1482.0


In [412]:
state_df = df['state'].value_counts()
state_df
df_corr.corr()

,brokered_by,price,bed,bath,acre_lot,street,zip_code,house_size
brokered_by,1.000000,-0.022056,0.050578,0.006557,-0.009066,0.019877,0.118156,0.003270
price,-0.022056,1.000000,0.142785,0.295196,-0.008105,-0.028503,-0.175699,0.182397
bed,0.050578,0.142785,1.000000,0.705189,0.003025,0.007267,0.142414,0.426493
bath,0.006557,0.295196,0.705189,1.000000,-0.008725,-0.006917,-0.046351,0.495834
acre_lot,-0.009066,-0.008105,0.003025,-0.008725,1.000000,-0.005711,-0.004564,-0.000698
street,0.019877,-0.028503,0.007267,-0.006917,-0.005711,1.000000,0.089927,0.006759
zip_code,0.118156,-0.175699,0.142414,-0.046351,-0.004564,0.089927,1.000000,-0.000787
house_size,0.003270,0.182397,0.426493,0.495834,-0.000698,0.006759,-0.000787,1.000000


In [413]:

le = LabelEncoder()
df['state_encoded'] = le.fit_transform(df['state'])
df['city_encoded'] = le.fit_transform(df['city'])
state_df = df[['state', 'state_encoded', 'city_encoded']]
state_df.head()
df.drop(['state', 'city', 'prev_sold_date'], axis = 1, inplace= True)
df.head()
df.dtypes

brokered_by      float64
price            float64
bed              float64
bath             float64
acre_lot         float64
street           float64
zip_code         float64
house_size       float64
state_encoded      int64
city_encoded       int64
dtype: object

In [414]:
df = df.dropna(subset = ['price', 'brokered_by', 'bed', 'bath', 'acre_lot', 'street', 'zip_code', 'house_size'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50723 entries, 5217 to 1523253
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   brokered_by    50723 non-null  float64
 1   price          50723 non-null  float64
 2   bed            50723 non-null  float64
 3   bath           50723 non-null  float64
 4   acre_lot       50723 non-null  float64
 5   street         50723 non-null  float64
 6   zip_code       50723 non-null  float64
 7   house_size     50723 non-null  float64
 8   state_encoded  50723 non-null  int64  
 9   city_encoded   50723 non-null  int64  
dtypes: float64(8), int64(2)
memory usage: 4.3 MB


In [415]:

print(df.isna().sum())
print(df.sum())

brokered_by      0
price            0
bed              0
bath             0
acre_lot         0
street           0
zip_code         0
house_size       0
state_encoded    0
city_encoded     0
dtype: int64
brokered_by      2.539527e+09
price            3.338001e+10
bed              1.834250e+05
bath             1.241490e+05
acre_lot         4.925288e+06
street           4.764837e+10
zip_code         6.430526e+08
house_size       1.064199e+08
state_encoded    0.000000e+00
city_encoded     4.747958e+07
dtype: float64


In [416]:


x_data = df.drop('price', axis =1 )
y_data = df['price']
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.2, random_state = 1)


In [417]:
lre = LinearRegression()
lre.fit(x_train, y_train)
lre.score(x_test, y_test)
lre.score(x_train, y_train)

0.22744689534456553